### USE CASE: Predicting what class of income the individual belong to (>50k or <=50k) using a Census Data

Reading the census_data.csv using pandas library

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('census_data.csv')

Understanding the Data Set

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
census.tail()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


Convert the Label column to 0s' and 1s' instead of strings

In [6]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [7]:
def label_fix(label):
    if label == '<=50k':
        return 0
    else:
        return 1

In [15]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)
c_s = census['income_bracket'].sort_values(ascending=True)
c_s.head()

0        1
21714    1
21713    1
21712    1
21711    1
Name: income_bracket, dtype: int64

Splitting data into Train and Test

In [17]:
from sklearn.model_selection import train_test_split

In [20]:
x_data = census.drop('income_bracket',axis=1)
y_labels = census['income_bracket']
X_train, X_tesy, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.35,random_state=101)

Tensorflow offers an API called estimator

model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

model.train(input_fn=input_func,steps=5000)

prepare feature columns and input function

In [21]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

Importing TensorFlow

In [2]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

Create the tf.feature_column for the categorical values. USe vocabulary list or just ise hash buckets.

In [ ]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
maritial_status = tf.feature_column.categorical_column_with_hash_bucket("maritial_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000) 
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

Create the continious feature_columns for the continious values using numeric_column

In [ ]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

Put all these variables into a single list with the variable name feat_cols

In [ ]:
feat_cols = [gender,occupation,maritial_status,relationship,education,workclass,native_country,age,education_num,capital_gain,capital_loss,hours_per_week]

Create the input function

Batch_size 

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=None,shuffle=True)

Create your model with tf.estimator

Create a LinearClassifier

In [ ]:
model = tf.estimatorLinearClassifier(feature_columns=feat_cols)

train your model on the data, for atleast 5000 steps

In [ ]:
model.train(input_fn=input_func,steps=5000)

Evaluation

Create a prediction input function. Provide shuffle=False

In [ ]:
predictions = list(model.predict(input_fn=pred_fn))

Each item in the list will loook like this:

In [ ]:
predictions[0]

Create a list of only the class_ids key values from the prediction list.

In [ ]:
final_pred = []
for pred in predictions:
    final_pred.append(pred['class_ids'][0])

In [ ]:
final_preds[:10]

Import classification report. To check model's performance

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, final_pred))

# Completed!